<a href="https://colab.research.google.com/github/PavelStelmakhV/hw309-selection-of-hyperparameters/blob/main/hw309.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [26]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from keras import layers
from keras import callbacks
from keras import initializers
from keras import regularizers

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import BatchNormalization

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
from tensorboard.plugins.hparams import api as hp

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical


# import numpy as np
# import matplotlib.pyplot as plt
# import tf_keras

# from keras import layers
# from keras import models
# from keras import regularizers
# from keras import callbacks
# from keras import initializers


# from keras.models import load_model


In [27]:
!rm -rf ./logs/

In [28]:
num_classes = 10
log_dir = 'logs/hparam_tuning/'

In [29]:
%%capture
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [30]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

In [31]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([512, 1024]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.3, 0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'adamax', 'nadam']))
METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [55]:
def layer_relu(model, neurons, drop_out):
  w_init_relu = initializers.HeNormal(seed=42)
  b_init = initializers.Zeros()
  l1 = 1e-3
  l2 = 1e-3
  model.add(BatchNormalization())
  model.add(layers.Dense(neurons,
                        activation='relu',
                        kernel_initializer=w_init_relu,
                        bias_initializer=b_init,
                        kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2),
                        bias_regularizer=regularizers.L1L2(l1=l1, l2=l2),
                        activity_regularizer=regularizers.L1L2(l1=l1, l2=l2)
                         )
  )
  model.add(layers.Dropout(drop_out))
  return model

In [56]:
def train_test_model(run_dir, hparams):

    # w_init_relu = initializers.HeNormal(seed=42)
    # b_init = initializers.Zeros()
    # l1 = 0.01
    # l2 = 0.01

    model = Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    model = layer_relu(model, hparams[HP_NUM_UNITS], hparams[HP_DROPOUT])
    model = layer_relu(model, hparams[HP_NUM_UNITS], hparams[HP_DROPOUT])
    model = layer_relu(model, hparams[HP_NUM_UNITS], hparams[HP_DROPOUT])
    model.add(layers.Dense(10, activation='softmax'))

    # model = tf.keras.models.Sequential([
    #     Flatten(input_shape=(28, 28)),
    #     BatchNormalization(),
    #     Dense(hparams[HP_NUM_UNITS], activation='relu', kernel_initializer=w_init_relu, bias_initializer=b_init,),
    #     Dropout(hparams[HP_DROPOUT]),
    #     BatchNormalization(),
    #     Dense(hparams[HP_NUM_UNITS], activation='relu', kernel_initializer=w_init_relu, bias_initializer=b_init),
    #     Dropout(hparams[HP_DROPOUT]),
    #     BatchNormalization(),
    #     Dense(hparams[HP_NUM_UNITS], activation='relu', kernel_initializer=w_init_relu, bias_initializer=b_init),
    #     Dropout(hparams[HP_DROPOUT]),
    #     Dense(10, activation='softmax')
    # ])
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='categorical_crossentropy', metrics=['accuracy'],
    )

    callback_list = [
        EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=6),
        ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True),
        TensorBoard(log_dir=run_dir, histogram_freq=1)
    ]
    # Train the model
    model.fit(x=x_train,
              y=y_train,
              epochs=100,
              batch_size=128,
              validation_split=0.13,
              callbacks=callback_list,
              verbose=0
              )

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    print(f"Test Accuracy: {test_accuracy}")
    return test_accuracy


In [57]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(run_dir, hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=2)

In [ ]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      run_name = run_name + f"[{optimizer}] [{dropout_rate}] [{num_units}]"
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})

      run(log_dir + run_name, hparams)
      session_num += 1

--- Starting trial: run-0[adam] [0.3] [512]
{'num_units': 512, 'dropout': 0.3, 'optimizer': 'adam'}

Epoch 1: val_accuracy improved from -inf to 0.76936, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_accuracy improved from 0.76936 to 0.79064, saving model to best_model.h5

Epoch 3: val_accuracy improved from 0.79064 to 0.79936, saving model to best_model.h5

Epoch 4: val_accuracy improved from 0.79936 to 0.80782, saving model to best_model.h5

Epoch 5: val_accuracy did not improve from 0.80782

Epoch 6: val_accuracy did not improve from 0.80782

Epoch 7: val_accuracy did not improve from 0.80782

Epoch 8: val_accuracy improved from 0.80782 to 0.81179, saving model to best_model.h5

Epoch 9: val_accuracy did not improve from 0.81179

Epoch 10: val_accuracy improved from 0.81179 to 0.81385, saving model to best_model.h5

Epoch 11: val_accuracy did not improve from 0.81385

Epoch 12: val_accuracy did not improve from 0.81385

Epoch 13: val_accuracy improved from 0.81385 to 0.81923, saving model to best_model.h5

Epoch 14: val_accuracy improved from 0.81923 to 0.82128, saving model to best_model.h5

Epoch 15: val_accuracy did not improve from 0.82128

Epoch 16: val_accura

In [ ]:
# test_loss, test_accuracy = model.evaluate(x_test, y_test)
# print(f"Test Accuracy: {test_accuracy}")

In [ ]:
# model.summary()

In [ ]:
%tensorboard --logdir logs/hparam_tuning